In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
from scipy import stats

from projimmo.data import *
from projimmo.preprocessor import *
from projimmo.model import *

In [2]:
query = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.DVF_{DATA_YEAR}
    """
#chemin d'accès à nos fichiers locaux
data_query_cache_path=Path(f"raw_data/valeursfoncieres-{DATA_YEAR}.txt")

# import de nos data, ici à partir du fichier mais plus tard dans la base de data de bigquer
df= get_data_with_cache(query=query,
gcp_project=GCP_PROJECT,
        cache_path=data_query_cache_path,
        data_has_header=True
   )

#Pour écrire dans big query le df notre choix
#load_data_to_bq(
#        df_de_notre_choix,
 #       gcp_project=GCP_PROJECT,
 #       bq_dataset=BQ_DATASET,
 #       table=f'processed_{DATA_SIZE}',
 #       truncate=True
  #  )
#appel à la fonction clean_data
df_0=clean_data(df)
#appel à la fonction clean_outliers
df_1=clean_outliers(df_0)
#on passe notre df dans la pipeline
df_2=preprocess_features(df_1)


Load data from local CSV...
✅ Data loaded, with shape (3727005, 43)
✅ X_processed, with shape (75664, 89)


In [3]:
df_2.head(20)

,surface_reelle_bati,surface_terrain,somme_surface_carrez,valeur_fonciere,code_type_local_2,type_de_voie_ACH,type_de_voie_ALL,type_de_voie_ART,type_de_voie_AV,type_de_voie_BD,...,type_de_voie_VOIE,type_de_voie_VTE,type_de_voie_ZAC,type_de_voie_ZI,type_de_voie_nan,nombre_pieces_principales,month_mutation,year_mutation,departement,nombre_de_lots
0,1.312723,0.0,1.215123,0.071429,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0
1,1.312723,0.0,1.135831,0.142857,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0
2,-0.374117,0.0,-0.169891,-0.607143,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0
3,0.064462,0.0,0.196049,-0.380952,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0
4,0.300619,0.0,0.302316,-0.357143,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0
5,1.245249,0.0,1.528474,-0.285714,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0
6,0.165672,0.0,0.384877,-0.642857,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0
7,1.481407,0.0,1.841008,0.071429,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0
8,0.806671,0.0,0.641553,-0.595238,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0
9,-0.272906,0.0,-0.383515,-0.490476,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0


In [6]:
X_train,X_test,y_train,y_test=split_df(df_2)
model_type="XGB" #possible: "KNR","LR","XGB"
model=initialize_model(model_type)
fitted_model=train_model(X_train,y_train,model)
metrics=evaluate_model(X_test,y_test,fitted_model, model_type)

Mean Squared Error (MSE): 1.0162283376437147
Mean Absolute Error (MAE): 0.4537000979069561
R-squared (R²) Score: 0.6644233933883885
Root Mean Squared Error (RMSE): 1.0080815133925007
Mean Squared Logarithmic Error (MSLE): None
